In [1]:

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
from ProcessTrueStateActionData import read_df_in_chunks

/usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


### Load and preprocess the data 
(produce tf train+test datasets)

In [2]:
train_test_split = 0.95

data_path = 'logs/APPO/TrueStates_200_1000_Meander_small/data'
states = np.load(data_path + '/states.npy')
states_t = np.load(data_path + '/states_t.npy')
afterstates = np.load(data_path + '/afterstates.npy')
next_states = np.load(data_path + '/next_states.npy')
actions = np.load(data_path + '/actions_onehot.npy')
rewards = np.load(data_path + '/rewards.npy')

states_actions = np.concatenate([states, actions], axis=1)
states_actions_t = np.concatenate([states_t, actions], axis=1)
next_states_delta = next_states - afterstates

afterstates = np.array(afterstates, dtype=np.float32)
next_states = np.array(next_states, dtype=np.float32)
states_actions = np.array(states_actions, dtype=np.float32)

divide = int(train_test_split * afterstates.shape[0])
afterstates_train = afterstates[:divide,:]
next_states_train = next_states[:divide,:]
afterstates_test = afterstates[divide:,:]
next_states_test = next_states[divide:,:]
print(afterstates.shape[0] - divide)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


9900


In [3]:
states_seq = np.load(data_path + '/states_seq.npy')
actions_seq = np.load(data_path + '/actions_onehot_seq.npy')

states_actions_seq = np.concatenate([states_seq, actions_seq], axis=2)

In [28]:
states_seq_20 = np.load(data_path + '/states_seq_20.npy')
actions_seq_20 = np.load(data_path + '/actions_onehot_seq_20.npy')

states_actions_seq_20 = np.concatenate([states_seq_20, actions_seq_20], axis=2)

In [ ]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(42+20,))
x = Dense(512, activation='relu')(input_)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(128, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='54_categorical_accuracy', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions, data_map, epochs=50, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=128)

In [15]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(43+20,))
x = Dense(512, activation='relu')(input_)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(128, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='54_categorical_accuracy', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions, data_map, epochs=50, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=128)

Epoch 1/50
1393/1393 [==============================] - 39s 22ms/step - loss: 2.2002 - 03_loss: 0.4389 - 04_loss: 0.0350 - 13_loss: 0.5983 - 14_loss: 0.4217 - 23_loss: 0.0027 - 24_loss: 0.0049 - 33_loss: 0.0037 - 34_loss: 0.0037 - 43_loss: 0.2849 - 44_loss: 0.0692 - 53_loss: 0.2842 - 54_loss: 0.0531 - 03_categorical_accuracy: 0.7069 - 04_categorical_accuracy: 0.9884 - 13_categorical_accuracy: 0.6726 - 14_categorical_accuracy: 0.7221 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 0.9993 - 33_categorical_accuracy: 0.9999 - 34_categorical_accuracy: 0.9997 - 43_categorical_accuracy: 0.9185 - 44_categorical_accuracy: 0.9790 - 53_categorical_accuracy: 0.9172 - 54_categorical_accuracy: 0.9844 - val_loss: 2.0139 - val_03_loss: 0.4172 - val_04_loss: 0.0164 - val_13_loss: 0.5810 - val_14_loss: 0.3888 - val_23_loss: 5.7768e-08 - val_24_loss: 1.0569e-07 - val_33_loss: 5.1079e-08 - val_34_loss: 1.8505e-07 - val_43_loss: 0.2674 - val_44_loss: 0.0376 - val_53_loss: 0.2791 - val_54_loss:

Epoch 7/50
1393/1393 [==============================] - 28s 20ms/step - loss: 1.9205 - 03_loss: 0.4028 - 04_loss: 0.0092 - 13_loss: 0.5656 - 14_loss: 0.3788 - 23_loss: 1.3486e-09 - 24_loss: 1.8764e-09 - 33_loss: 1.4851e-09 - 34_loss: 2.1213e-09 - 43_loss: 0.2630 - 44_loss: 0.0254 - 53_loss: 0.2606 - 54_loss: 0.0151 - 03_categorical_accuracy: 0.7191 - 04_categorical_accuracy: 0.9951 - 13_categorical_accuracy: 0.6887 - 14_categorical_accuracy: 0.7338 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9222 - 44_categorical_accuracy: 0.9880 - 53_categorical_accuracy: 0.9229 - 54_categorical_accuracy: 0.9926 - val_loss: 1.8907 - val_03_loss: 0.3977 - val_04_loss: 0.0082 - val_13_loss: 0.5588 - val_14_loss: 0.3750 - val_23_loss: 1.1439e-09 - val_24_loss: 1.3426e-09 - val_33_loss: 1.2041e-09 - val_34_loss: 1.5714e-09 - val_43_loss: 0.2528 - val_44_loss: 0.0239 - val_53_loss: 0.261

Epoch 13/50
1393/1393 [==============================] - 28s 20ms/step - loss: 1.8777 - 03_loss: 0.3965 - 04_loss: 0.0076 - 13_loss: 0.5571 - 14_loss: 0.3735 - 23_loss: 5.2179e-10 - 24_loss: 5.9939e-10 - 33_loss: 5.7129e-10 - 34_loss: 7.2583e-10 - 43_loss: 0.2546 - 44_loss: 0.0227 - 53_loss: 0.2524 - 54_loss: 0.0133 - 03_categorical_accuracy: 0.7272 - 04_categorical_accuracy: 0.9960 - 13_categorical_accuracy: 0.6970 - 14_categorical_accuracy: 0.7411 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9240 - 44_categorical_accuracy: 0.9888 - 53_categorical_accuracy: 0.9252 - 54_categorical_accuracy: 0.9931 - val_loss: 1.8315 - val_03_loss: 0.3883 - val_04_loss: 0.0069 - val_13_loss: 0.5443 - val_14_loss: 0.3658 - val_23_loss: 4.3349e-10 - val_24_loss: 5.1176e-10 - val_33_loss: 4.3951e-10 - val_34_loss: 5.6594e-10 - val_43_loss: 0.2426 - val_44_loss: 0.0215 - val_53_loss: 0.25

Epoch 19/50
1393/1393 [==============================] - 29s 21ms/step - loss: 1.8398 - 03_loss: 0.3901 - 04_loss: 0.0066 - 13_loss: 0.5471 - 14_loss: 0.3686 - 23_loss: 2.4952e-10 - 24_loss: 2.8230e-10 - 33_loss: 2.3614e-10 - 34_loss: 2.9702e-10 - 43_loss: 0.2478 - 44_loss: 0.0215 - 53_loss: 0.2454 - 54_loss: 0.0126 - 03_categorical_accuracy: 0.7348 - 04_categorical_accuracy: 0.9965 - 13_categorical_accuracy: 0.7031 - 14_categorical_accuracy: 0.7483 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9253 - 44_categorical_accuracy: 0.9892 - 53_categorical_accuracy: 0.9268 - 54_categorical_accuracy: 0.9934 - val_loss: 1.7821 - val_03_loss: 0.3807 - val_04_loss: 0.0057 - val_13_loss: 0.5293 - val_14_loss: 0.3594 - val_23_loss: 1.4450e-10 - val_24_loss: 1.2041e-10 - val_33_loss: 1.8062e-10 - val_34_loss: 1.6256e-10 - val_43_loss: 0.2340 - val_44_loss: 0.0199 - val_53_loss: 0.24

Epoch 25/50
1393/1393 [==============================] - 29s 21ms/step - loss: 1.8093 - 03_loss: 0.3850 - 04_loss: 0.0059 - 13_loss: 0.5385 - 14_loss: 0.3644 - 23_loss: 1.8263e-10 - 24_loss: 1.9601e-10 - 33_loss: 1.5921e-10 - 34_loss: 2.1340e-10 - 43_loss: 0.2425 - 44_loss: 0.0209 - 53_loss: 0.2401 - 54_loss: 0.0120 - 03_categorical_accuracy: 0.7388 - 04_categorical_accuracy: 0.9969 - 13_categorical_accuracy: 0.7085 - 14_categorical_accuracy: 0.7515 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9268 - 44_categorical_accuracy: 0.9896 - 53_categorical_accuracy: 0.9274 - 54_categorical_accuracy: 0.9936 - val_loss: 1.7503 - val_03_loss: 0.3743 - val_04_loss: 0.0052 - val_13_loss: 0.5189 - val_14_loss: 0.3548 - val_23_loss: 1.3848e-10 - val_24_loss: 1.7460e-10 - val_33_loss: 1.1439e-10 - val_34_loss: 1.9266e-10 - val_43_loss: 0.2286 - val_44_loss: 0.0198 - val_53_loss: 0.23

Epoch 31/50
1393/1393 [==============================] - 28s 20ms/step - loss: 1.7899 - 03_loss: 0.3812 - 04_loss: 0.0056 - 13_loss: 0.5328 - 14_loss: 0.3619 - 23_loss: 8.3620e-11 - 24_loss: 9.4324e-11 - 33_loss: 7.8938e-11 - 34_loss: 9.7000e-11 - 43_loss: 0.2390 - 44_loss: 0.0204 - 53_loss: 0.2373 - 54_loss: 0.0117 - 03_categorical_accuracy: 0.7443 - 04_categorical_accuracy: 0.9971 - 13_categorical_accuracy: 0.7121 - 14_categorical_accuracy: 0.7547 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9270 - 44_categorical_accuracy: 0.9895 - 53_categorical_accuracy: 0.9281 - 54_categorical_accuracy: 0.9938 - val_loss: 1.7279 - val_03_loss: 0.3683 - val_04_loss: 0.0048 - val_13_loss: 0.5123 - val_14_loss: 0.3516 - val_23_loss: 6.6227e-11 - val_24_loss: 1.4450e-10 - val_33_loss: 5.4186e-11 - val_34_loss: 7.2248e-11 - val_43_loss: 0.2257 - val_44_loss: 0.0197 - val_53_loss: 0.23

Epoch 37/50
1393/1393 [==============================] - 28s 20ms/step - loss: 1.7732 - 03_loss: 0.3781 - 04_loss: 0.0053 - 13_loss: 0.5282 - 14_loss: 0.3597 - 23_loss: 6.4220e-11 - 24_loss: 8.4289e-11 - 33_loss: 5.9538e-11 - 34_loss: 6.9572e-11 - 43_loss: 0.2362 - 44_loss: 0.0199 - 53_loss: 0.2343 - 54_loss: 0.0116 - 03_categorical_accuracy: 0.7467 - 04_categorical_accuracy: 0.9972 - 13_categorical_accuracy: 0.7140 - 14_categorical_accuracy: 0.7566 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9273 - 44_categorical_accuracy: 0.9897 - 53_categorical_accuracy: 0.9287 - 54_categorical_accuracy: 0.9938 - val_loss: 1.7091 - val_03_loss: 0.3645 - val_04_loss: 0.0046 - val_13_loss: 0.5063 - val_14_loss: 0.3487 - val_23_loss: 9.6331e-11 - val_24_loss: 1.2041e-10 - val_33_loss: 1.2643e-10 - val_34_loss: 9.6331e-11 - val_43_loss: 0.2231 - val_44_loss: 0.0194 - val_53_loss: 0.23

Epoch 43/50
1393/1393 [==============================] - 29s 21ms/step - loss: 1.7590 - 03_loss: 0.3756 - 04_loss: 0.0050 - 13_loss: 0.5244 - 14_loss: 0.3574 - 23_loss: 3.4786e-11 - 24_loss: 4.0807e-11 - 33_loss: 3.0772e-11 - 34_loss: 3.7462e-11 - 43_loss: 0.2338 - 44_loss: 0.0196 - 53_loss: 0.2319 - 54_loss: 0.0113 - 03_categorical_accuracy: 0.7488 - 04_categorical_accuracy: 0.9974 - 13_categorical_accuracy: 0.7153 - 14_categorical_accuracy: 0.7577 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9279 - 44_categorical_accuracy: 0.9899 - 53_categorical_accuracy: 0.9291 - 54_categorical_accuracy: 0.9940 - val_loss: 1.6814 - val_03_loss: 0.3613 - val_04_loss: 0.0043 - val_13_loss: 0.5000 - val_14_loss: 0.3455 - val_23_loss: 3.6124e-11 - val_24_loss: 4.2145e-11 - val_33_loss: 3.0103e-11 - val_34_loss: 2.4083e-11 - val_43_loss: 0.2170 - val_44_loss: 0.0181 - val_53_loss: 0.22

Epoch 49/50
1393/1393 [==============================] - 29s 21ms/step - loss: 1.7494 - 03_loss: 0.3734 - 04_loss: 0.0049 - 13_loss: 0.5211 - 14_loss: 0.3558 - 23_loss: 2.3414e-11 - 24_loss: 2.6759e-11 - 33_loss: 2.6090e-11 - 34_loss: 2.5421e-11 - 43_loss: 0.2327 - 44_loss: 0.0194 - 53_loss: 0.2308 - 54_loss: 0.0113 - 03_categorical_accuracy: 0.7503 - 04_categorical_accuracy: 0.9974 - 13_categorical_accuracy: 0.7185 - 14_categorical_accuracy: 0.7603 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9280 - 44_categorical_accuracy: 0.9900 - 53_categorical_accuracy: 0.9290 - 54_categorical_accuracy: 0.9940 - val_loss: 1.6699 - val_03_loss: 0.3581 - val_04_loss: 0.0039 - val_13_loss: 0.4969 - val_14_loss: 0.3439 - val_23_loss: 3.0103e-11 - val_24_loss: 5.4186e-11 - val_33_loss: 3.6124e-11 - val_34_loss: 6.0207e-11 - val_43_loss: 0.2160 - val_44_loss: 0.0180 - val_53_loss: 0.22

In [16]:
model.save_weights('AfterStateModel_MulitLabel')

In [17]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input, GRU
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(10,42+20,))
#x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_)
#x = Bidirectional(GRU(64))(input_)
x = Bidirectional(LSTM(256))(input_)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(256, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_seq_10, data_map, epochs=1000, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

Epoch 1/1000
697/697 [==============================] - 33s 28ms/step - loss: 4.4017 - 03_loss: 0.6320 - 04_loss: 0.1742 - 13_loss: 0.7749 - 14_loss: 0.6859 - 23_loss: 0.0075 - 24_loss: 0.0098 - 33_loss: 0.0086 - 34_loss: 0.0116 - 43_loss: 0.2909 - 44_loss: 1.1848 - 53_loss: 0.2882 - 54_loss: 0.3334 - 03_categorical_accuracy: 0.6740 - 04_categorical_accuracy: 0.9536 - 13_categorical_accuracy: 0.6525 - 14_categorical_accuracy: 0.6521 - 23_categorical_accuracy: 0.9997 - 24_categorical_accuracy: 0.9998 - 33_categorical_accuracy: 0.9986 - 34_categorical_accuracy: 0.9986 - 43_categorical_accuracy: 0.9180 - 44_categorical_accuracy: 0.4837 - 53_categorical_accuracy: 0.9151 - 54_categorical_accuracy: 0.9053 - val_loss: 4.2701 - val_03_loss: 0.6273 - val_04_loss: 0.1551 - val_13_loss: 0.7657 - val_14_loss: 0.6649 - val_23_loss: 3.1649e-07 - val_24_loss: 5.8154e-07 - val_33_loss: 4.9701e-07 - val_34_loss: 6.2476e-07 - val_43_loss: 0.2765 - val_44_loss: 1.1928 - val_53_loss: 0.2754 - val_54_loss:

Epoch 7/1000
697/697 [==============================] - 18s 25ms/step - loss: 3.9383 - 03_loss: 0.6069 - 04_loss: 0.1279 - 13_loss: 0.7551 - 14_loss: 0.6435 - 23_loss: 3.2298e-09 - 24_loss: 5.2681e-09 - 33_loss: 5.8648e-09 - 34_loss: 8.5306e-09 - 43_loss: 0.2672 - 44_loss: 1.0422 - 53_loss: 0.2657 - 54_loss: 0.2298 - 03_categorical_accuracy: 0.6791 - 04_categorical_accuracy: 0.9644 - 13_categorical_accuracy: 0.6571 - 14_categorical_accuracy: 0.6708 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9208 - 44_categorical_accuracy: 0.5451 - 53_categorical_accuracy: 0.9186 - 54_categorical_accuracy: 0.9263 - val_loss: 4.3786 - val_03_loss: 0.6140 - val_04_loss: 0.1520 - val_13_loss: 0.7606 - val_14_loss: 0.6674 - val_23_loss: 9.0310e-11 - val_24_loss: 1.0837e-10 - val_33_loss: 6.6227e-11 - val_34_loss: 5.4186e-11 - val_43_loss: 0.2670 - val_44_loss: 1.2680 - val_53_loss: 0.273

Epoch 13/1000
697/697 [==============================] - 17s 25ms/step - loss: 3.5563 - 03_loss: 0.5785 - 04_loss: 0.0989 - 13_loss: 0.7328 - 14_loss: 0.6027 - 23_loss: 4.6694e-10 - 24_loss: 8.3487e-10 - 33_loss: 6.1745e-10 - 34_loss: 8.5494e-10 - 43_loss: 0.2469 - 44_loss: 0.8885 - 53_loss: 0.2482 - 54_loss: 0.1597 - 03_categorical_accuracy: 0.6966 - 04_categorical_accuracy: 0.9699 - 13_categorical_accuracy: 0.6625 - 14_categorical_accuracy: 0.6870 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9243 - 44_categorical_accuracy: 0.6110 - 53_categorical_accuracy: 0.9223 - 54_categorical_accuracy: 0.9438 - val_loss: 4.8900 - val_03_loss: 0.6352 - val_04_loss: 0.1811 - val_13_loss: 0.7766 - val_14_loss: 0.7131 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.2841 - val_44_loss: 1.5154 - val_53_loss: 0.28

Epoch 19/1000
697/697 [==============================] - 17s 24ms/step - loss: 3.2394 - 03_loss: 0.5388 - 04_loss: 0.0827 - 13_loss: 0.6965 - 14_loss: 0.5701 - 23_loss: 8.8771e-10 - 24_loss: 1.5032e-09 - 33_loss: 9.6264e-10 - 34_loss: 1.4329e-09 - 43_loss: 0.2189 - 44_loss: 0.7848 - 53_loss: 0.2214 - 54_loss: 0.1261 - 03_categorical_accuracy: 0.7221 - 04_categorical_accuracy: 0.9728 - 13_categorical_accuracy: 0.6706 - 14_categorical_accuracy: 0.7019 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9286 - 44_categorical_accuracy: 0.6555 - 53_categorical_accuracy: 0.9272 - 54_categorical_accuracy: 0.9506 - val_loss: 5.4171 - val_03_loss: 0.6901 - val_04_loss: 0.2058 - val_13_loss: 0.8189 - val_14_loss: 0.7604 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.3177 - val_44_loss: 1.7290 - val_53_loss: 0.32

Epoch 25/1000
697/697 [==============================] - 17s 25ms/step - loss: 2.9760 - 03_loss: 0.4979 - 04_loss: 0.0738 - 13_loss: 0.6536 - 14_loss: 0.5423 - 23_loss: 3.0237e-10 - 24_loss: 4.6961e-10 - 33_loss: 3.2712e-10 - 34_loss: 5.2982e-10 - 43_loss: 0.1936 - 44_loss: 0.7133 - 53_loss: 0.1944 - 54_loss: 0.1071 - 03_categorical_accuracy: 0.7467 - 04_categorical_accuracy: 0.9745 - 13_categorical_accuracy: 0.6866 - 14_categorical_accuracy: 0.7169 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9329 - 44_categorical_accuracy: 0.6844 - 53_categorical_accuracy: 0.9329 - 54_categorical_accuracy: 0.9547 - val_loss: 5.8858 - val_03_loss: 0.7609 - val_04_loss: 0.2280 - val_13_loss: 0.8743 - val_14_loss: 0.8155 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.3528 - val_44_loss: 1.8508 - val_53_loss: 0.37

...

Epoch 127/1000
697/697 [==============================] - 17s 24ms/step - loss: 1.7625 - 03_loss: 0.3004 - 04_loss: 0.0573 - 13_loss: 0.3527 - 14_loss: 0.3121 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.1045 - 44_loss: 0.4609 - 53_loss: 0.1041 - 54_loss: 0.0705 - 03_categorical_accuracy: 0.8419 - 04_categorical_accuracy: 0.9783 - 13_categorical_accuracy: 0.8227 - 14_categorical_accuracy: 0.8349 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9524 - 44_categorical_accuracy: 0.7743 - 53_categorical_accuracy: 0.9525 - 54_categorical_accuracy: 0.9623 - val_loss: 9.5119 - val_03_loss: 1.1810 - val_04_loss: 0.3431 - val_13_loss: 1.5665 - val_14_loss: 1.4145 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5587 - val_44_loss: 2.8205 - val_53_loss: 0.5

Epoch 133/1000
697/697 [==============================] - 17s 24ms/step - loss: 1.7472 - 03_loss: 0.2978 - 04_loss: 0.0570 - 13_loss: 0.3496 - 14_loss: 0.3091 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.1034 - 44_loss: 0.4571 - 53_loss: 0.1034 - 54_loss: 0.0699 - 03_categorical_accuracy: 0.8424 - 04_categorical_accuracy: 0.9784 - 13_categorical_accuracy: 0.8226 - 14_categorical_accuracy: 0.8354 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9530 - 44_categorical_accuracy: 0.7748 - 53_categorical_accuracy: 0.9529 - 54_categorical_accuracy: 0.9629 - val_loss: 9.6780 - val_03_loss: 1.2052 - val_04_loss: 0.3576 - val_13_loss: 1.5775 - val_14_loss: 1.4047 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5641 - val_44_loss: 2.8666 - val_53_loss: 0.5

Epoch 139/1000
697/697 [==============================] - 17s 24ms/step - loss: 1.7324 - 03_loss: 0.2954 - 04_loss: 0.0568 - 13_loss: 0.3460 - 14_loss: 0.3049 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.1031 - 44_loss: 0.4548 - 53_loss: 0.1023 - 54_loss: 0.0691 - 03_categorical_accuracy: 0.8433 - 04_categorical_accuracy: 0.9785 - 13_categorical_accuracy: 0.8242 - 14_categorical_accuracy: 0.8366 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9528 - 44_categorical_accuracy: 0.7747 - 53_categorical_accuracy: 0.9526 - 54_categorical_accuracy: 0.9632 - val_loss: 9.8265 - val_03_loss: 1.1967 - val_04_loss: 0.3698 - val_13_loss: 1.5953 - val_14_loss: 1.4347 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5733 - val_44_loss: 2.9716 - val_53_loss: 0.5

Epoch 145/1000
697/697 [==============================] - 16s 23ms/step - loss: 1.7194 - 03_loss: 0.2934 - 04_loss: 0.0568 - 13_loss: 0.3420 - 14_loss: 0.3023 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.1023 - 44_loss: 0.4517 - 53_loss: 0.1018 - 54_loss: 0.0691 - 03_categorical_accuracy: 0.8432 - 04_categorical_accuracy: 0.9785 - 13_categorical_accuracy: 0.8251 - 14_categorical_accuracy: 0.8371 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9532 - 44_categorical_accuracy: 0.7762 - 53_categorical_accuracy: 0.9532 - 54_categorical_accuracy: 0.9630 - val_loss: 9.9256 - val_03_loss: 1.2195 - val_04_loss: 0.3661 - val_13_loss: 1.6150 - val_14_loss: 1.4415 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5815 - val_44_loss: 2.9993 - val_53_loss: 0.6


KeyboardInterrupt



In [30]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input, GRU
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
        
input_ = Input(shape=(10,42+20,))
#x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_)
#x = Bidirectional(GRU(64))(input_)
x = Bidirectional(LSTM(256))(input_)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(256, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_seq_10, data_map, epochs=1000, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

Epoch 1/1000
697/697 [==============================] - 32s 28ms/step - loss: 4.3925 - 03_loss: 0.6321 - 04_loss: 0.1746 - 13_loss: 0.7745 - 14_loss: 0.6850 - 23_loss: 0.0075 - 24_loss: 0.0088 - 33_loss: 0.0069 - 34_loss: 0.0112 - 43_loss: 0.2902 - 44_loss: 1.1840 - 53_loss: 0.2869 - 54_loss: 0.3309 - 03_categorical_accuracy: 0.6734 - 04_categorical_accuracy: 0.9531 - 13_categorical_accuracy: 0.6536 - 14_categorical_accuracy: 0.6531 - 23_categorical_accuracy: 0.9997 - 24_categorical_accuracy: 0.9998 - 33_categorical_accuracy: 0.9996 - 34_categorical_accuracy: 0.9986 - 43_categorical_accuracy: 0.9182 - 44_categorical_accuracy: 0.4848 - 53_categorical_accuracy: 0.9147 - 54_categorical_accuracy: 0.9054 - val_loss: 4.2576 - val_03_loss: 0.6201 - val_04_loss: 0.1571 - val_13_loss: 0.7662 - val_14_loss: 0.6684 - val_23_loss: 1.2516e-07 - val_24_loss: 3.7638e-07 - val_33_loss: 1.2050e-07 - val_34_loss: 1.7278e-07 - val_43_loss: 0.2729 - val_44_loss: 1.1825 - val_53_loss: 0.2771 - val_54_loss:

Epoch 7/1000
697/697 [==============================] - 16s 23ms/step - loss: 3.9325 - 03_loss: 0.6065 - 04_loss: 0.1270 - 13_loss: 0.7547 - 14_loss: 0.6422 - 23_loss: 5.6581e-09 - 24_loss: 7.4636e-09 - 33_loss: 5.7243e-09 - 34_loss: 6.0782e-09 - 43_loss: 0.2671 - 44_loss: 1.0405 - 53_loss: 0.2657 - 54_loss: 0.2289 - 03_categorical_accuracy: 0.6796 - 04_categorical_accuracy: 0.9648 - 13_categorical_accuracy: 0.6575 - 14_categorical_accuracy: 0.6712 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9209 - 44_categorical_accuracy: 0.5459 - 53_categorical_accuracy: 0.9180 - 54_categorical_accuracy: 0.9264 - val_loss: 4.3855 - val_03_loss: 0.6122 - val_04_loss: 0.1547 - val_13_loss: 0.7617 - val_14_loss: 0.6675 - val_23_loss: 1.3264e-08 - val_24_loss: 1.3516e-08 - val_33_loss: 1.3270e-08 - val_34_loss: 1.3288e-08 - val_43_loss: 0.2665 - val_44_loss: 1.2742 - val_53_loss: 0.273

Epoch 13/1000
697/697 [==============================] - 17s 24ms/step - loss: 3.5486 - 03_loss: 0.5763 - 04_loss: 0.0987 - 13_loss: 0.7320 - 14_loss: 0.6009 - 23_loss: 6.1411e-10 - 24_loss: 8.2751e-10 - 33_loss: 7.1847e-10 - 34_loss: 7.5860e-10 - 43_loss: 0.2463 - 44_loss: 0.8873 - 53_loss: 0.2474 - 54_loss: 0.1598 - 03_categorical_accuracy: 0.6986 - 04_categorical_accuracy: 0.9699 - 13_categorical_accuracy: 0.6626 - 14_categorical_accuracy: 0.6879 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9243 - 44_categorical_accuracy: 0.6109 - 53_categorical_accuracy: 0.9225 - 54_categorical_accuracy: 0.9441 - val_loss: 4.9405 - val_03_loss: 0.6374 - val_04_loss: 0.1833 - val_13_loss: 0.7783 - val_14_loss: 0.7121 - val_23_loss: 0.0000e+00 - val_24_loss: 1.3245e-08 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.2841 - val_44_loss: 1.5606 - val_53_loss: 0.29

Epoch 19/1000
697/697 [==============================] - 16s 24ms/step - loss: 3.2270 - 03_loss: 0.5352 - 04_loss: 0.0823 - 13_loss: 0.6934 - 14_loss: 0.5675 - 23_loss: 5.9003e-10 - 24_loss: 9.8806e-10 - 33_loss: 6.1009e-10 - 34_loss: 7.3987e-10 - 43_loss: 0.2179 - 44_loss: 0.7849 - 53_loss: 0.2201 - 54_loss: 0.1257 - 03_categorical_accuracy: 0.7243 - 04_categorical_accuracy: 0.9729 - 13_categorical_accuracy: 0.6717 - 14_categorical_accuracy: 0.7037 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9289 - 44_categorical_accuracy: 0.6548 - 53_categorical_accuracy: 0.9279 - 54_categorical_accuracy: 0.9515 - val_loss: 5.4896 - val_03_loss: 0.6924 - val_04_loss: 0.2118 - val_13_loss: 0.8204 - val_14_loss: 0.7711 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.3202 - val_44_loss: 1.7568 - val_53_loss: 0.32

Epoch 25/1000
697/697 [==============================] - 17s 24ms/step - loss: 2.9590 - 03_loss: 0.4950 - 04_loss: 0.0742 - 13_loss: 0.6478 - 14_loss: 0.5380 - 23_loss: 1.7962e-09 - 24_loss: 2.3795e-09 - 33_loss: 2.0216e-09 - 34_loss: 2.1748e-09 - 43_loss: 0.1915 - 44_loss: 0.7111 - 53_loss: 0.1933 - 54_loss: 0.1081 - 03_categorical_accuracy: 0.7482 - 04_categorical_accuracy: 0.9744 - 13_categorical_accuracy: 0.6890 - 14_categorical_accuracy: 0.7193 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9338 - 44_categorical_accuracy: 0.6859 - 53_categorical_accuracy: 0.9332 - 54_categorical_accuracy: 0.9548 - val_loss: 5.9889 - val_03_loss: 0.7594 - val_04_loss: 0.2366 - val_13_loss: 0.8893 - val_14_loss: 0.8190 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.3650 - val_44_loss: 1.8814 - val_53_loss: 0.36

...

Epoch 169/1000
697/697 [==============================] - 16s 23ms/step - loss: 1.6696 - 03_loss: 0.2848 - 04_loss: 0.0563 - 13_loss: 0.3299 - 14_loss: 0.2924 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0990 - 44_loss: 0.4403 - 53_loss: 0.0998 - 54_loss: 0.0671 - 03_categorical_accuracy: 0.8459 - 04_categorical_accuracy: 0.9789 - 13_categorical_accuracy: 0.8281 - 14_categorical_accuracy: 0.8399 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9539 - 44_categorical_accuracy: 0.7780 - 53_categorical_accuracy: 0.9533 - 54_categorical_accuracy: 0.9637 - val_loss: 10.3398 - val_03_loss: 1.2187 - val_04_loss: 0.4106 - val_13_loss: 1.6781 - val_14_loss: 1.4924 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5977 - val_44_loss: 3.1269 - val_53_loss: 0.

Epoch 175/1000
697/697 [==============================] - 16s 23ms/step - loss: 1.6605 - 03_loss: 0.2834 - 04_loss: 0.0561 - 13_loss: 0.3271 - 14_loss: 0.2908 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0986 - 44_loss: 0.4390 - 53_loss: 0.0989 - 54_loss: 0.0666 - 03_categorical_accuracy: 0.8458 - 04_categorical_accuracy: 0.9789 - 13_categorical_accuracy: 0.8286 - 14_categorical_accuracy: 0.8404 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9537 - 44_categorical_accuracy: 0.7788 - 53_categorical_accuracy: 0.9538 - 54_categorical_accuracy: 0.9640 - val_loss: 10.4106 - val_03_loss: 1.2508 - val_04_loss: 0.4100 - val_13_loss: 1.6888 - val_14_loss: 1.5113 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5896 - val_44_loss: 3.1466 - val_53_loss: 0.

Epoch 181/1000
697/697 [==============================] - 16s 23ms/step - loss: 1.6524 - 03_loss: 0.2818 - 04_loss: 0.0562 - 13_loss: 0.3261 - 14_loss: 0.2896 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0979 - 44_loss: 0.4357 - 53_loss: 0.0985 - 54_loss: 0.0666 - 03_categorical_accuracy: 0.8473 - 04_categorical_accuracy: 0.9789 - 13_categorical_accuracy: 0.8292 - 14_categorical_accuracy: 0.8406 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9542 - 44_categorical_accuracy: 0.7801 - 53_categorical_accuracy: 0.9538 - 54_categorical_accuracy: 0.9641 - val_loss: 10.6028 - val_03_loss: 1.2481 - val_04_loss: 0.4045 - val_13_loss: 1.7103 - val_14_loss: 1.5330 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5931 - val_44_loss: 3.2200 - val_53_loss: 0.

Epoch 187/1000
697/697 [==============================] - 17s 24ms/step - loss: 1.6436 - 03_loss: 0.2806 - 04_loss: 0.0559 - 13_loss: 0.3243 - 14_loss: 0.2868 - 23_loss: 0.0000e+00 - 24_loss: 0.0000e+00 - 33_loss: 0.0000e+00 - 34_loss: 0.0000e+00 - 43_loss: 0.0975 - 44_loss: 0.4341 - 53_loss: 0.0984 - 54_loss: 0.0660 - 03_categorical_accuracy: 0.8462 - 04_categorical_accuracy: 0.9788 - 13_categorical_accuracy: 0.8292 - 14_categorical_accuracy: 0.8413 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9541 - 44_categorical_accuracy: 0.7807 - 53_categorical_accuracy: 0.9538 - 54_categorical_accuracy: 0.9642 - val_loss: 10.7346 - val_03_loss: 1.2458 - val_04_loss: 0.4301 - val_13_loss: 1.7058 - val_14_loss: 1.5301 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.5967 - val_44_loss: 3.2698 - val_53_loss: 0.

KeyboardInterrupt: 

In [31]:
model.save_weights('NextStateModel_MulitLabel_rnn')

In [18]:
labels, encoding, counts = np.unique(rewards, return_inverse=True, return_counts=True)
reward_encoding = np.eye(labels.shape[0])[encoding]
reward_map = {}
for i in range(labels.shape[0]):
    reward_map[i] = labels[i]
reward_map

{0: -2.0999999046325684,
 1: -2.0,
 2: -1.100000023841858,
 3: -1.0,
 4: -0.10000000149011612,
 5: 0.0}

In [24]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input, GRU
from keras.layers import Bidirectional
from keras.models import Model

NUM_NODES = 6
NODE_CLASSES = [3, 4]

data_map = {}
losses = []
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        data_map[str(i)+str(n)] = next_states[:,index:index+n]
        losses.append(tf.keras.losses.CategoricalCrossentropy())
        index += n
data_map['reward'] = reward_encoding
losses.append(tf.keras.losses.CategoricalCrossentropy())
        
input_ = Input(shape=(10,42+20,))
#x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(input_)
#x = Bidirectional(GRU(64))(input_)
x = Bidirectional(LSTM(256))(input_)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)

outs = []
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        x_ = Dense(256, activation='relu')(x)
        outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))
        
x_ = Dense(256, activation='relu')(x)
outs.append(Dense(6, activation='softmax', name='reward')(x_))

model = Model(input_, outs)
model.compile(optimizer='adam', loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions_seq_10, data_map, epochs=1000, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=256)

Epoch 1/1000
697/697 [==============================] - 33s 30ms/step - loss: 5.4668 - 03_loss: 0.6315 - 04_loss: 0.1764 - 13_loss: 0.7750 - 14_loss: 0.6847 - 23_loss: 0.0091 - 24_loss: 0.0102 - 33_loss: 0.0082 - 34_loss: 0.0116 - 43_loss: 0.2902 - 44_loss: 1.1849 - 53_loss: 0.2876 - 54_loss: 0.3327 - reward_loss: 1.0648 - 03_categorical_accuracy: 0.6743 - 04_categorical_accuracy: 0.9527 - 13_categorical_accuracy: 0.6532 - 14_categorical_accuracy: 0.6530 - 23_categorical_accuracy: 0.9983 - 24_categorical_accuracy: 0.9994 - 33_categorical_accuracy: 0.9990 - 34_categorical_accuracy: 0.9991 - 43_categorical_accuracy: 0.9180 - 44_categorical_accuracy: 0.4843 - 53_categorical_accuracy: 0.9158 - 54_categorical_accuracy: 0.9049 - reward_categorical_accuracy: 0.5433 - val_loss: 5.2932 - val_03_loss: 0.6189 - val_04_loss: 0.1552 - val_13_loss: 0.7654 - val_14_loss: 0.6659 - val_23_loss: 4.7474e-07 - val_24_loss: 3.8696e-07 - val_33_loss: 4.1844e-07 - val_34_loss: 3.2508e-07 - val_43_loss: 0.276

697/697 [==============================] - 17s 25ms/step - loss: 4.9745 - 03_loss: 0.6098 - 04_loss: 0.1323 - 13_loss: 0.7572 - 14_loss: 0.6470 - 23_loss: 1.5386e-08 - 24_loss: 1.1673e-08 - 33_loss: 9.6090e-09 - 34_loss: 8.9534e-09 - 43_loss: 0.2691 - 44_loss: 1.0723 - 53_loss: 0.2671 - 54_loss: 0.2411 - reward_loss: 0.9786 - 03_categorical_accuracy: 0.6777 - 04_categorical_accuracy: 0.9637 - 13_categorical_accuracy: 0.6562 - 14_categorical_accuracy: 0.6705 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9204 - 44_categorical_accuracy: 0.5320 - 53_categorical_accuracy: 0.9176 - 54_categorical_accuracy: 0.9235 - reward_categorical_accuracy: 0.5701 - val_loss: 5.4483 - val_03_loss: 0.6142 - val_04_loss: 0.1530 - val_13_loss: 0.7604 - val_14_loss: 0.6675 - val_23_loss: 2.6611e-08 - val_24_loss: 1.3294e-08 - val_33_loss: 1.3306e-08 - val_34_loss: 1.3294e-08 - val_43_loss: 0.

697/697 [==============================] - 18s 26ms/step - loss: 4.5792 - 03_loss: 0.5893 - 04_loss: 0.1080 - 13_loss: 0.7422 - 14_loss: 0.6129 - 23_loss: 3.2793e-09 - 24_loss: 3.0639e-09 - 33_loss: 2.7341e-09 - 34_loss: 2.8859e-09 - 43_loss: 0.2552 - 44_loss: 0.9415 - 53_loss: 0.2553 - 54_loss: 0.1744 - reward_loss: 0.9004 - 03_categorical_accuracy: 0.6904 - 04_categorical_accuracy: 0.9684 - 13_categorical_accuracy: 0.6609 - 14_categorical_accuracy: 0.6834 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9229 - 44_categorical_accuracy: 0.5877 - 53_categorical_accuracy: 0.9210 - 54_categorical_accuracy: 0.9406 - reward_categorical_accuracy: 0.5929 - val_loss: 5.9003 - val_03_loss: 0.6250 - val_04_loss: 0.1753 - val_13_loss: 0.7681 - val_14_loss: 0.7008 - val_23_loss: 1.3245e-08 - val_24_loss: 1.3245e-08 - val_33_loss: 1.3245e-08 - val_34_loss: 1.3245e-08 - val_43_loss: 0.

697/697 [==============================] - 18s 26ms/step - loss: 4.2385 - 03_loss: 0.5596 - 04_loss: 0.0908 - 13_loss: 0.7175 - 14_loss: 0.5851 - 23_loss: 1.8330e-09 - 24_loss: 2.1427e-09 - 33_loss: 1.8350e-09 - 34_loss: 1.9855e-09 - 43_loss: 0.2345 - 44_loss: 0.8436 - 53_loss: 0.2360 - 54_loss: 0.1372 - reward_loss: 0.8344 - 03_categorical_accuracy: 0.7092 - 04_categorical_accuracy: 0.9715 - 13_categorical_accuracy: 0.6656 - 14_categorical_accuracy: 0.6958 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9262 - 44_categorical_accuracy: 0.6306 - 53_categorical_accuracy: 0.9245 - 54_categorical_accuracy: 0.9486 - reward_categorical_accuracy: 0.6145 - val_loss: 6.4747 - val_03_loss: 0.6597 - val_04_loss: 0.1913 - val_13_loss: 0.7934 - val_14_loss: 0.7422 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.

697/697 [==============================] - 18s 25ms/step - loss: 3.9530 - 03_loss: 0.5271 - 04_loss: 0.0805 - 13_loss: 0.6863 - 14_loss: 0.5629 - 23_loss: 2.7354e-09 - 24_loss: 3.0906e-09 - 33_loss: 2.5059e-09 - 34_loss: 2.9354e-09 - 43_loss: 0.2123 - 44_loss: 0.7727 - 53_loss: 0.2145 - 54_loss: 0.1167 - reward_loss: 0.7800 - 03_categorical_accuracy: 0.7290 - 04_categorical_accuracy: 0.9734 - 13_categorical_accuracy: 0.6740 - 14_categorical_accuracy: 0.7066 - 23_categorical_accuracy: 1.0000 - 24_categorical_accuracy: 1.0000 - 33_categorical_accuracy: 1.0000 - 34_categorical_accuracy: 1.0000 - 43_categorical_accuracy: 0.9298 - 44_categorical_accuracy: 0.6608 - 53_categorical_accuracy: 0.9283 - 54_categorical_accuracy: 0.9533 - reward_categorical_accuracy: 0.6341 - val_loss: 6.9721 - val_03_loss: 0.7081 - val_04_loss: 0.2158 - val_13_loss: 0.8288 - val_14_loss: 0.7809 - val_23_loss: 0.0000e+00 - val_24_loss: 0.0000e+00 - val_33_loss: 0.0000e+00 - val_34_loss: 0.0000e+00 - val_43_loss: 0.

KeyboardInterrupt: 

In [ ]:
probs[0]

In [14]:
probs = model.predict(states_actions[0:1])

state = np.zeros(42)
index_state = 0
index = 0
for i in range(NUM_NODES):
    for n in NODE_CLASSES:
        state[index_state+np.random.choice(np.arange(n), p=probs[index][0])] = 1
        index_state += n
        index += 1
state

1/1 [==============================] - 0s 51ms/step


array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1.])

In [42]:
prediction = model.predict(np.array([states_actions[50]]))
print(prediction)

new_state = {}

for i in range(100):
    state = np.zeros(42, dtype=np.int8)
    index_state, index = 0, 0
    for i in range(NUM_NODES):
        for n in NODE_CLASSES:
            state[index_state+np.random.choice(np.arange(n), p=prediction[index][0])] = 1
            index_state += n; index += 1
    if not state.tobytes() in new_state:
        new_state[state.tobytes()] = 1
    else:
        new_state[state.tobytes()] += 1
        
ns = {k: v for k, v in sorted(new_state.items(), key=lambda item: item[1], reverse=True)}
            
for s in ns.keys():
    print(ns[s])
    print(np.frombuffer(s, dtype="int8"))

print(len(new_state.keys()))

1/1 [==============================] - 0s 29ms/step
[array([[9.9999988e-01, 2.8316990e-10, 6.2747894e-08]], dtype=float32), array([[1.4892132e-35, 9.9999988e-01, 3.8243346e-08, 1.2682605e-07]],
      dtype=float32), array([[2.6334348e-08, 6.1443985e-02, 9.3855602e-01]], dtype=float32), array([[0.00808789, 0.7319339 , 0.02746691, 0.23251131]], dtype=float32), array([[1.3898515e-15, 1.0057768e-15, 1.0000000e+00]], dtype=float32), array([[4.0497008e-16, 5.7430210e-16, 4.9967082e-16, 1.0000000e+00]],
      dtype=float32), array([[4.0957807e-16, 3.2186437e-16, 1.0000000e+00]], dtype=float32), array([[7.2782153e-16, 8.9383671e-16, 1.0721520e-15, 1.0000000e+00]],
      dtype=float32), array([[3.632320e-08, 2.473178e-02, 9.752681e-01]], dtype=float32), array([[1.01864416e-07, 3.16646350e-07, 3.63653725e-12, 9.99999523e-01]],
      dtype=float32), array([[2.7840000e-10, 4.2507645e-02, 9.5749241e-01]], dtype=float32), array([[3.4164978e-09, 2.7237392e-13, 1.2582257e-14, 1.0000000e+00]],
      dt

In [38]:
target = states_actions[50]
new_state = {}

for i in range(194040):
    if (target==states_actions[i]).all():
        state = np.array(next_states[i], dtype=np.int8)
        if not state.tobytes() in new_state:
            new_state[state.tobytes()] = 1
        else:
            new_state[state.tobytes()] += 1
        
ns = {k: v for k, v in sorted(new_state.items(), key=lambda item: item[1], reverse=True)}

count = 0
for s in ns.keys():
    print(ns[s])
    count+=ns[s]
    print(np.frombuffer(s, dtype="int8"))

print(len(new_state.keys()))
print(count)

58
[1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
19
[1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
5
[1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1
 0 0 0 0 1]
4
[1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
4
[1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
2
[1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
2
[1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0
 1 0 0 0 1]
1
[1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0
 1 0 0 0 1]
1
[1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 1]
9
96


In [4]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from keras.models import Model


model = Sequential()
#model.add(Bidirectional(LSTM(256, activation='relu', return_sequences=True), input_shape=(sequence_length, encoding_len)))
#model.add(Flatten())
model.add(Input(shape=(42,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.001)

print(model.summary())

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(next_states, rewards, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=64)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 256)               11008     
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dense_6 (Dense)             (None, 256)               65792     
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 142,849
Trainable params: 142,849
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/250
2785/2785 [==============================] - 12s 4ms/step - loss: 0.2694 - val_loss: 0.2778
Epoch 2/250
2785/2785 [==============================] - 10s 4ms/step -


KeyboardInterrupt



In [5]:
model.save_weights('Reward_Model_Regression')

In [18]:
labels, encoding, counts = np.unique(rewards, return_inverse=True, return_counts=True)
encoding = np.eye(labels.shape[0])[encoding]
reward_map = {}
for i in range(labels.shape[0]):
    reward_map[i] = labels[i]
reward_map

{0: -2.0999999046325684,
 1: -2.0,
 2: -1.100000023841858,
 3: -1.0,
 4: -0.10000000149011612,
 5: 0.0}

In [64]:
model = Sequential()
model.add(Input(shape=(42+20,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

print(model.summary())

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_actions, encoding, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=64)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_72 (Dense)            (None, 512)               32256     
                                                                 
 dense_73 (Dense)            (None, 512)               262656    
                                                                 
 dense_74 (Dense)            (None, 512)               262656    
                                                                 
 dense_75 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 560,646
Trainable params: 560,646
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/250
2785/2785 [==============================] - 13s 4ms/step - loss: 0.5015 - categorical_accuracy: 0.8336 - val_loss: 0.4996 - val_categorical_accuracy: 0.8299
Epoch


KeyboardInterrupt



In [21]:
model = Sequential()
model.add(Input(shape=(43,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7)

from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.0005)

print(model.summary())

#model.fit(train_dataset,epochs=250, verbose=1, callbacks=[callback])

model.fit(states_t, encoding, epochs=250, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=64)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 512)               22528     
                                                                 
 dense_41 (Dense)            (None, 512)               262656    
                                                                 
 dense_42 (Dense)            (None, 512)               262656    
                                                                 
 dense_43 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 550,918
Trainable params: 550,918
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/250
2785/2785 [==============================] - 15s 5ms/step - loss: 0.4082 - categorical_accuracy: 0.8771 - val_loss: 0.3842 - val_categorical_accuracy: 0.8820
Epoch

KeyboardInterrupt: 

In [20]:
model.save_weights('RewardModel')
np.save('reward_map.npy', reward_map) 

In [25]:
target = states[53]
rewards_ = {}

for i in range(194040):
    if (target==states[i]).all():
        reward = rewards[i]
        if not reward in rewards_:
            rewards_[reward] = 1
        else:
            rewards_[reward] += 1
        
ns = {k: v for k, v in sorted(rewards_.items(), key=lambda item: item[1], reverse=True)}

count = 0
for s in ns.keys():
    print(ns[s])
    count+=ns[s]
    print(s)



11334
-1.0
528
-1.100000023841858
43
-2.0


In [67]:
target = states_actions[53]
rewards_ = {}

for i in range(194040):
    if (target==states_actions[i]).all():
        reward = rewards[i-1]
        if not reward in rewards_:
            rewards_[reward] = 1
        else:
            rewards_[reward] += 1
        
ns = {k: v for k, v in sorted(rewards_.items(), key=lambda item: item[1], reverse=True)}

count = 0
for s in ns.keys():
    print(ns[s])
    count+=ns[s]
    print(s)

print(len(new_state.keys()))
print(count)

59
-1.0
10
0.0
6
-1.100000023841858
5
-2.0
1
80
